In [48]:
import numpy as np
import tensorflow as tf
import os

load training data from .dat files
raw_data_imaginary_x.dat --> train_imag
raw_data_real_x.dat      --> train_real

In [49]:
# if IS_TRAINING == True, a new model will be compiled 
# and trained from the dataset
#
# if IS_TRAINING == False, an existing model will be 
# loaded and tested against the dataset
IS_TRAINING = True

# trainingset to be trained or loaded
trainingset_number = 5

# training params
neuron_number = 8
activation_function = 'tanh'
iterations = 20000
learning_rate= 1e-4

checkpoint_path = "./training_" + str(trainingset_number) + "/cp_test.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)



In [50]:
filename_raw_data_real = "raw_data_real_" + str(trainingset_number) + ".dat"
filename_raw_data_imag = "raw_data_imaginary_" + str(trainingset_number) + ".dat"

filename_dataset_csv = "training_data_" + str(trainingset_number) + ".csv"

# my_file = open(filename_raw_data_real, "r")
# lines_real = my_file.readlines()
# my_file.close()

# my_file = open(filename_raw_data_imag, "r")
# lines_imag = my_file.readlines()
# my_file.close()

# train_real = []
# train_imag = []

# for line in lines_real:
#     train_real = [train_real ,float(line)]

# for line in lines_imag:
#     train_imag = [train_imag ,float(line)]

# dataset_real = tf.data.TextLineDataset(filename_raw_data_real)
# dataset_imag = tf.data.TextLineDataset(filename_raw_data_imag)

# # convert strings to float
# def parse_fnc(line):
#     string_vals = tf.strings.split([line]).values
#     return tf.strings.to_number(string_vals, tf.float32)

# dataset_real = dataset_real.map(map_func=parse_fnc)
# dataset_imag = dataset_imag.map(map_func=parse_fnc)

# # merge datasets
# dataset = dataset_real.zip(dataset_imag)

dataset = tf.data.experimental.CsvDataset(
  filename_dataset_csv,
  [tf.float32,  # Required field, use dtype or empty tensor
   tf.float32,
   tf.float32,
   tf.float32
  ],
  header=True,
  select_cols=[0,1,2,3]  
)




In [51]:
print(dataset)
i = 0

for element in dataset.as_numpy_iterator():
  print(element)
  i = i + 1
  if i > 10:
      break

<CsvDatasetV2 element_spec=(TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>
(0.09036217, -0.2613757, 0.22294177, 0.1604512)
(0.022515569, -0.1132093, 0.16743372, 0.06361104)
(0.114548385, -0.100870565, 0.12522025, -0.011140993)
(0.049135324, -0.014699027, 0.098613776, -0.06175121)
(-0.13331392, -0.12705529, 0.087487355, -0.089262664)
(0.1544656, -0.3077073, 0.089055814, -0.09597649)
(0.11328178, -0.026650935, 0.09796924, -0.08606317)
(0.27299157, -0.015948214, 0.10823633, -0.06591184)
(0.15798786, 0.18548791, 0.11556848, -0.042117316)
(0.11445405, 0.01783486, 0.11829203, -0.018676687)
(0.06248361, -0.12510371, 0.116487764, 0.0037648869)


In [52]:
if IS_TRAINING:
    FC_layer = tf.keras.layers.Dense(units=neuron_number, activation=activation_function) # dense layer == fully connected layer

    input_layer = tf.keras.layers.Dense(units=2, input_dim=2, activation=activation_function)
    output_layer = tf.keras.layers.Dense(units=2, activation=activation_function)

In [53]:
# create the model
if IS_TRAINING:
    model = tf.keras.Sequential([input_layer, FC_layer, output_layer])


In [54]:
# loss function
# as described in article: MSE mean squared error
# batches have the format:
# [batch_real, batch_imag]
def my_loss_fcn(ground_truth_batch, predicted_batch):
    result = tf.square(predicted_batch[:,1] - ground_truth_batch[:,1])
    result = result + tf.square(predicted_batch[:,2] - ground_truth_batch[:,2])
    N = len(ground_truth_batch)
    result = tf.reduce_sum(result,1)
    return result  

In [55]:
# compile the keras model
# loss=tf.keras.losses.MeanSquaredError()
if IS_TRAINING:
    model.compile(loss=my_loss_fcn, 
    optimizer=tf.keras.optimizers.Adam(learning_rate), 
    metrics=['accuracy',
    tf.keras.metrics.RootMeanSquaredError()])    

In [56]:
# load model from file if not training
if not IS_TRAINING:
    model = tf.keras.models.load_model("./training_" + str(trainingset_number) + "/saved_trained_model")

In [57]:
# Display the model's architecture
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 2)                 6         
                                                                 
 dense_12 (Dense)            (None, 8)                 24        
                                                                 
 dense_14 (Dense)            (None, 2)                 18        
                                                                 
Total params: 48
Trainable params: 48
Non-trainable params: 0
_________________________________________________________________


In [58]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [59]:

# fit the keras model on the dataset
#
# train the model by slicing the data into "batches" of size batch_size,
# and repeatedly iterating over the entire dataset 
# for a given number of epochs
if IS_TRAINING:
    history = model.fit(dataset, 
    epochs=iterations)

Epoch 1/20000


ValueError: in user code:

    File "c:\Users\jonas\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\jonas\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jonas\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\jonas\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 856, in train_step
        x, y, sample_weight = data_adapter.unpack_x_y_sample_weight(data)
    File "c:\Users\jonas\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py", line 1577, in unpack_x_y_sample_weight
        raise ValueError(error_ms

In [ ]:
# test the dataset against the loaded model
if not IS_TRAINING:
    model.evaluate()

In [ ]:
# save the model in the correct directory
if IS_TRAINING:
    model.save("./training_" + str(trainingset_number) + "/saved_trained_model")

In [ ]:
# The returned history object holds a record of the loss values 
# and metric values during training
print(history.history)

In [ ]:
# training progress
# diagram number 2 paper page 4

# cost function mean square error